In [ ]:
import pandas as pd
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
sox_data = yf.download("^SOX", start="2022-06-30", end="2024-06-27", interval="1d")['Close']
print(sox_data.head(20))
print(len(sox_data))

In [ ]:
sox_data = sox_data.pct_change().shift(1)
sox_data

In [ ]:
file_path = 'C:/Users/A51607/Desktop/交易競賽/更新的/testData.xlsx'
data = pd.read_excel(file_path)
data['date'] = data['datetime'].dt.date
data['time'] = data['datetime'].dt.time
data['time'] = pd.to_datetime(data['time'], format='%H:%M:%S').dt.time
data.head(30)

In [ ]:
data_mor = data[(data['time'] >= pd.to_datetime('08:45:00').time()) & (data['time'] <= pd.to_datetime('13:15:00').time())]
date_list = []
open_list = []
high_list = []
low_list = []
close_list = []
volume_list = []

for i in range(0, len(data_mor), 10):
    chunk = data_mor.iloc[i:i+10]
    date_list.append(chunk['date'].iloc[0])
    open_list.append(chunk['open'].iloc[0])
    high_list.append(chunk['high'].max())
    low_list.append(chunk['low'].min())
    close_list.append(chunk['close'].iloc[-1])
    volume_list.append(chunk['volume'].sum())

daily_data = pd.DataFrame({'Date': date_list, 'Open': open_list, 'High': high_list, 'Low': low_list, 'Close': close_list, 'Volume': volume_list})
daily_data.set_index('Date', inplace=True)

print(daily_data.head(20))
print(len(daily_data))

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(sox_data['Close'], label='SOX Index', color='blue')
plt.title('SOX Index')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)

In [ ]:
plt.figure(figsize=(14, 7))
plt.plot(daily_data['Close'], label='TX Future', color='red')
plt.title('Taiwan Index Future')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)

In [ ]:
date_series = pd.to_datetime(date_list)
previous_day_series = date_series - pd.Timedelta(days=1)
previous_business_days = [date - pd.offsets.BDay(1) for date in date_series]
previous_business_days_str = [date.strftime('%Y-%m-%d') for date in previous_business_days]
print(previous_business_days_str)

In [ ]:
all_dates = pd.DataFrame(index=pd.to_datetime(previous_business_days_str))
sox_predate = all_dates.join(sox_data, how='left')
sox_predate = sox_predate.rename(columns={'Open': 'S-Open', 'High': 'S-High', 'Low': 'S-Low', 'Close': 'S-Close'})
print(sox_predate)

In [ ]:
f_close = daily_data['Close'].to_list()
s_close = sox_predate['S-Close'].to_list()
merged_df = pd.DataFrame({'Date': date_list, 'F-Close': f_close, 'S-Close': s_close})
print(merged_df.head(20))
comparison = merged_df.dropna()
future_price = pd.Series(comparison['F-Close'])
sox_price = pd.Series(comparison['S-Close'])
correlation = future_price.corr(sox_price)
print(f'SOX & Future Correlation: {correlation}')